News2TopicsWordCloud

- ニュース記事をテキスト分析する手段として、トピックモデル(LDA)を適用
- 得られたトピックそれぞれについて、ワードクラウドを作成

In [1]:
import os
import random
import json
import requests
import pandas as pd
import numpy as np
# topic modeling
from janome.tokenizer import Tokenizer
from gensim.corpora.dictionary import Dictionary
from gensim.models import LdaModel
# pyLDAvis
import pyLDAvis
import pyLDAvis.gensim_models as gensimvis
# word cloud
from wordcloud import WordCloud
import matplotlib.pyplot as plt
import japanize_matplotlib
ttf_path = japanize_matplotlib.get_font_ttf_path()

import warnings
warnings.filterwarnings("ignore")

## JSONデータを読み込み

crisis.yuiseki.net のクローリングしたニュース記事の一覧を取得するAPIから、JSONをとってきました。

JSONデータは次のような中身が入っています。

In [2]:
## データを取得
response_json = []
for i in range(100):
    json_url = 'https://crisis.yuiseki.net/api/news?country=%E6%97%A5%E6%9C%AC&category=poverty&limit=1000'
    json_url += f'&page={i+1}'
    print(i)
    response = requests.get(json_url)
    _response_json = response.json()
    if len(_response_json) == 0:
        break
    response_json += _response_json   

0
1


In [3]:
## 1レコードの中身
response_json[0]

{'tags': ['poverty', 'drug_abuse'],
 '_id': '61012e712659a5387a2e32f3',
 'url': 'https://www.fnn.jp/articles/-/216600',
 '__v': 0,
 'category': 'poverty',
 'createdAt': '2021-07-28T10:16:17.111Z',
 'latitude': 36.5748441,
 'longitude': 139.2394179,
 'ogDesc': '「国立競技場の運営ボランティア向け弁当が消費期限前に大量に廃棄されている」との一部報道が波紋を広げている。大会組織委員会は28日の会見で「余剰により多くの会場で食品ロスが生じていたことを確認した」と事実を認めた一方で、「余剰は廃棄ではなくリサイクルだ」と強調した。日本では7人に1人の子どもが貧困家庭で育ち、満足な食事を取れていないと言われる。「廃棄される弁当を困窮する子どもに」と要望するNPOを取材した。\xa0オリパラの大会組織委員会（以下組織委）は28日の会見で、運営ボランティア向け弁当の大量廃…',
 'ogImage': 'https://fnn.ismcdn.jp/mwimgs/0/1/600wm/img_0159bbea4bbed75a3cea857f830a0a57124373.jpg',
 'ogTitle': '「捨てる弁当は子どもに配って」弁当4000食分廃棄問題でNPOが組織委員会に異例の要望',
 'ogUrl': 'https://www.fnn.jp/articles/-/216600',
 'placeCity': None,
 'placeCountry': '日本',
 'placePref': None,
 'sourceConfirmed': True,
 'sourceName': 'フジテレビ',
 'sourceType': '放送局/キー局',
 'title': '「捨てる弁当は子どもに配って」弁当4000食分廃棄問題でNPOが組織委員会に異例の要望',
 'updatedAt': '2021-07-28T10:16:17.111Z'}

In [4]:
## データフレーム化
df = pd.DataFrame(response_json)

In [5]:
len(df)

861

## タグの出現頻度

ニュース(n=860)に含まれるタグを集計しました。

「貧困(poverty)」のほか、「ウイルス(virus)」「支援(support)」「ひとり親(poverty_one_parent)」などが目立ちます。

In [6]:
## タグについて
tags = []
for _tags in df['tags'].values:
    for tag in _tags:
        tags.append(tag)
tags = pd.Series(tags)

In [7]:
## どんなタグがよくニュースに使われているのか？
cnt = tags.value_counts().to_frame(name="頻度")
cnt["出現%"] = (cnt["頻度"] / len(df) * 100).round(2)
cnt.head(10)

,頻度,出現%
poverty,685,79.56
poverty_tech_intern,163,18.93
virus,163,18.93
support,151,17.54
poverty_food,99,11.50
poverty_child,91,10.57
poverty_non_regular,70,8.13
poverty_one_parent,50,5.81
poverty_woman,50,5.81
politics,49,5.69


## Preprocessing

LDAにかける準備として、タイトルと概要を合わせたものをテキストとみなして、形態素解析で単語分割をしました。

１文字の形態素はノイズが多いと感じたので「てにをは」のストップワードを兼ねて取り除きました。

前処理後のテキストには、タグの情報も含めました。

In [8]:
## text_corpus
text_corpus = (df['ogTitle'] + "\n\n" + df['ogDesc']).values

In [9]:
print(text_corpus[0])

「捨てる弁当は子どもに配って」弁当4000食分廃棄問題でNPOが組織委員会に異例の要望

「国立競技場の運営ボランティア向け弁当が消費期限前に大量に廃棄されている」との一部報道が波紋を広げている。大会組織委員会は28日の会見で「余剰により多くの会場で食品ロスが生じていたことを確認した」と事実を認めた一方で、「余剰は廃棄ではなくリサイクルだ」と強調した。日本では7人に1人の子どもが貧困家庭で育ち、満足な食事を取れていないと言われる。「廃棄される弁当を困窮する子どもに」と要望するNPOを取材した。 オリパラの大会組織委員会（以下組織委）は28日の会見で、運営ボランティア向け弁当の大量廃…


In [10]:
## processed_texts
t = Tokenizer()

processed_texts = []
for (idx, row), text in zip(df.iterrows(), text_corpus):
    # break
    if pd.isna(text):
        continue
    
    processed_text = []
    
    # タイトル・概要をトークン化
    tokens = t.tokenize(text)
    for token in tokens:
        surface = token.surface.strip()
        if len(surface) <= 1:
            continue
        processed_text.append(surface)
    
    # タグ
    processed_text += [ 'tag:' + tag for tag in row['tags'] ]
    
    processed_texts.append(processed_text)

In [11]:
print(processed_texts[0])

['捨てる', '弁当', '子ども', '配っ', '弁当', '4000', '食分', '廃棄', '問題', 'NPO', '組織', '委員', '異例', '要望', '国立', '競技', '運営', 'ボランティア', '向け', '弁当', '消費', '期限', '大量', '廃棄', 'いる', '一部', '報道', '波紋', '広げ', 'いる', '大会', '組織', '委員', '28', '会見', '余剰', 'により', '多く', '会場', '食品', 'ロス', '生じ', 'こと', '確認', '事実', '認め', '一方', '余剰', '廃棄', 'なく', 'リサイクル', '強調', '日本', '子ども', '貧困', '家庭', '育ち', '満足', '食事', '取れ', 'ない', '言わ', 'れる', '廃棄', 'れる', '弁当', '困窮', 'する', '子ども', '要望', 'する', 'NPO', '取材', 'オリパラ', '大会', '組織', '委員', '以下', '組織', '28', '会見', '運営', 'ボランティア', '向け', '弁当', '大量', 'tag:poverty', 'tag:drug_abuse']


## LDA model

前処理済みテキストで、LDAのモデルを学習しました。

In [12]:
random.seed(42)
np.random.seed(7)
os.environ['PYTHONHASHSEED'] = '42'

In [13]:
## gensim dictionary
dictionary = Dictionary(processed_texts)
corpus = [dictionary.doc2bow(text) for text in processed_texts]

In [14]:
# Train LDA model.

# Set training parameters.
num_topics = 30
chunksize = 2000
passes = 20
iterations = 400
eval_every = None  # Don't evaluate model perplexity, takes too much time.

# Make a index to word dictionary.
temp = dictionary[0]  # This is only to "load" the dictionary.
id2word = dictionary.id2token

model = LdaModel(
    corpus=corpus,
    id2word=id2word,
    chunksize=chunksize,
    alpha='auto',
    eta='auto',
    iterations=iterations,
    num_topics=num_topics,
    passes=passes,
    eval_every=eval_every,
)

## pyLDAvis

可視化ツール pyLDAvis で学習結果を可視化しました。

画面右の単語のヒストグラムでは、水色部分は全体の単語頻度、赤色部分はトピックの推定単語頻度を示しています。

λはソート用のパラメータで、0に近いほど「トピックに関連性の高い単語」を、1に近いほど「トピックにおける単語の頻度」を重視してソートします。

結果はかっこいいのですが、解釈が難しく、結論には貢献しませんでした。

In [15]:
## pyLDAvis
pyLDAvis.enable_notebook()
lda_viz = gensimvis.prepare(model, corpus, dictionary)
lda_viz

PreparedData(topic_coordinates=              x         y  topics  cluster       Freq
topic                                                
10     0.127296  0.206096       1        1  10.512263
6     -0.169103  0.002819       2        1   8.852946
25     0.063124 -0.055958       3        1   5.812473
13    -0.203669  0.084160       4        1   4.757810
5      0.025739 -0.038926       5        1   4.647024
27     0.048747 -0.037199       6        1   4.277638
7     -0.075039 -0.026425       7        1   4.189204
0      0.082470 -0.045570       8        1   4.169499
29     0.117064  0.117549       9        1   3.790334
23     0.028227 -0.053145      10        1   3.584451
22     0.016802 -0.023563      11        1   3.249797
26     0.030734  0.017673      12        1   3.156998
11    -0.130082  0.030977      13        1   3.062286
24     0.003365 -0.068147      14        1   3.048805
19    -0.107247 -0.026987      15        1   2.916748
2      0.061835 -0.047236      16        1   2.837605
15     0.013550 -0.066592      17        1   2.652009
17     0.061233  0.030902      18        1   2.638887
16     0.058935  0.031998      19        1   2.564438
4     -0.067212 -0.015972      20        1   2.539730
3      0.027243 -0.049146      21        1   2.234791
1      0.022887 -0.069325      22        1   2.195865
21    -0.054096 -0.068029      23        1   1.917207
8     -0.042168  0.161925      24        1   1.809109
14     0.032301 -0.012512      25        1   1.790026
28     0.057834 -0.065751      26        1   1.566535
18     0.057237  0.027324      27        1   1.448645
9     -0.084370  0.052530      28        1   1.338773
12     0.012891  0.023435      29        1   1.333981
20    -0.016530 -0.016905      30        1   1.104125, topic_info=     Term        Freq       Total Category  logprob  loglift
194    実習  391.000000  391.000000  Default  30.0000  30.0000
204    技能  336.000000  336.000000  Default  29.0000  29.0000
369    生理  136.000000  136.000000  Default  28.0000  28.0000
344    労働  166.000000  166.000000  Default  27.0000  27.0000
258    外国  183.000000  183.000000  Default  26.0000  26.0000
..    ...         ...         ...      ...      ...      ...
7      する    4.638146  451.760782  Topic30  -4.7661  -0.0727
264    対象    2.696547   51.503830  Topic30  -5.3084   1.5564
41     日本    3.742673  504.168983  Topic30  -4.9806  -0.3970
169  ニュース    3.013879  156.667803  Topic30  -5.1971   0.5552
5      いる    2.846864  394.153920  Topic30  -5.2542  -0.4244

[2100 rows x 6 columns], token_table=      Topic      Freq  Term
term                       
8030     18  0.609652    !!
607       1  0.168193   ...
607       2  0.079150   ...
607       3  0.029681   ...
607       4  0.148406   ...
...     ...       ...   ...
1920      2  0.132754  ＳＤＧｓ
1920     10  0.132754  ＳＤＧｓ
1920     24  0.132754  ＳＤＧｓ
1920     26  0.265507  ＳＤＧｓ
1541     11  0.893008   ＷＢＯ

[5991 rows x 3 columns], R=30, lambda_step=0.01, plot_opts={'xlab': 'PC1', 'ylab': 'PC2'}, topic_order=[11, 7, 26, 14, 6, 28, 8, 1, 30, 24, 23, 27, 12, 25, 20, 3, 16, 18, 17, 5, 4, 2, 22, 9, 15, 29, 19, 10, 13, 21])

## ワードクラウド

学習された lda model のそれぞれのトピックについて、ワードクラウド化を行いました。

In [16]:
nrows = 6
ncols = 5
fig, axes = plt.subplots(nrows=nrows, ncols=ncols,figsize=(12, 10))

for i in range(ncols):
    for j in range(nrows):
        ax  = axes[j][i]
        idx = i + ncols * j
        # word cloud
        topic_id = idx
        topic_repr = model.show_topic(topicid=topic_id, topn=30)
        wc = WordCloud(background_color="white",font_path=ttf_path, width=600, height=400, random_state=42)
        # plot
        ax.imshow(wc.fit_words(dict(topic_repr)))
        ax.set_title(f'{topic_id=}')
        ax.axis('off')

## ワードクラウドの解釈

トピックごとのワードクラウドの観察より（途中で疲れたので完全ではありませんが）、以下のようなキーワードとトピックの関係が抽出されました。

```
技能実習生 id=1 id=19 id=24
片親 id=2
シングルマザー id=3
外国人留学生 id=5
子どもの貧困 id=7
高齢者 id=9
生理用品 id=10
人身売買問題 id=11
男性の貧困 id=12
待機児童 id=17
学生 id=29
```

ワードクラウドの作業を通じて、貧困の大雑把なカテゴリ分けには、たとえば以下のキーワードが利用できるのではないかと思いました。

```
老後の生活、シングルマザー、技能実習生、外国人留学生、
子供、生理用品、失職している男性、ベトナム人、生活保護
```